In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import Pandas as pd
from Ipython.display import display
from scipy.stats import linregress

T = np.array([1,2,3,4,5,6,7,8,9,10]) # tiempo segundos 
T_min = T/60 # tiempo minutos
SF-1 = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]) 
m_h20 = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]) # masa de agua en g
W = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]) # gh20/gsep 
w0 = 0.1 # gh20/gsep #! cambiar valor
we  = 0.075
Di = 0.0676 # m²
De = 8/1000 # m de espesor 
L = 0.25 # m de longitud de bandeja
A = 0.0271 # m² de area de bandeja
mb = 275*2 # g de masa de bandeja
msepiolita = 300 # g de masa de sepiolita
Temp = np.array([25,25,25,25,25,25,25,25,25,25]) + 273.15 # K
plt.plot(T_min, W, 'ro')
plt.xlabel('Tiempo (min)')
plt.ylabel('W')
plt.title('W vs Tiempo')
plt.legend()
plt.show()


Delta_w = lambda W: W[1:] - W[:-1]
Delta_t = lambda T: T_min[1:] - T_min[:-1]
dW_dt = Delta_w(W) / Delta_t(T)

df = pd.DataFrame({
    'Tiempo (min)': T_min[1:],  # Usar T_min desde el segundo valor
    'dW_dt': dW_dt
})

display(df)
window_size = 3
rolling_mean = pd.Series(dW_dt).rolling(window=window_size, center=True).mean()

plt.plot(T[1:], dW_dt, label='dW/dt (original)', marker='o')
plt.plot(T[1:], df['dW_dt_rolling'], label=f'dW/dt (media móvil, ventana={window_size})', linestyle='--')
plt.xlabel('Tiempo (s)')
plt.ylabel('dW/dt')
plt.title('dW/dt y su media móvil')
plt.legend()
plt.show()

tol = 1e-3
diff = rolling_mean.diff().abs()
constant = diff < tol


# Extraer el valor constante
if constant.any():  # Si hay al menos un tramo constante
    rc = rolling_mean[constant].iloc[0]  # velocidad de secado en el periodo de velocidad constante
    print(f"Valor constante encontrado: {rc}")
else:
    rc = None
    print("No se encontró ningún tramo constante.")

index = df[df['dW_dt']< 0].index
for idx in index:
    tin = df.loc[idx, 'Tiempo (min)']
    Wc = W[idx+1]
tc =(w0-Wc)/rc  # tiempo critico

# calculo difusividad efectiva 
Defect = np.log(((W-we)/(Wc-we)))
slope, intercept, r_value, p_value, std_err = linregress(T_min, Defect)
r_squared = r_value**2
df_difusividad = pd.DataFrame({
    'Tiempo (min)': T_min,  
    'Defect': Defect
})
display(df_difusividad)
plt.plot(T_min, Defect, 'ro')
plt.plot(T_min, slope * T_min + intercept, 'b-', label=f'Ajuste lineal: $R^2$={r_squared:.3f}')
plt.xlabel('Tiempo (min)')
plt.ylabel('Defect')
plt.title('Defect vs Tiempo')
plt.legend()
plt.show()

#! meter Tg  que es la media de temperatura del aire ST-4 durante la etapa de velocidad de secado decreciente
#! es decir despues del punto -dw/dt < 0 

Tp = Temp[0] 

# Dg = 5.02e-9 *((Tp+Tg)/2)**1.5
#mu = 4.25e8*Tg+5.87e-6  # viscosidad en pa*s
#rho = 353.33/Tg # densidad en kg/m^3
#delta = De - 
# Demp = slope/np.pi**2/ (4*(delta/1000)**2) # m^2/s